In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

In [2]:
kernel

In [3]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [4]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

In [5]:
context = kernel.create_new_context()
context["history"] = ""

In [6]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Hello! I'd be happy to help you find some book suggestions. Can you please tell me what genre or specific topics you're interested in?


In [7]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hello! I'd be happy to help you find some book suggestions. Can you please tell me what genre or specific topics you're interested in?



In [8]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [9]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Great! Based on your interests in history, philosophy, and Greece, I have a few book suggestions for you:

1. "The Histories" by Herodotus - This classic work is often considered the first work of history in Western literature. It covers the history of the ancient world, including Greece, Egypt, and Persia.

2. "The Peloponnesian War" by Thucydides - This historical account covers the conflict between Athens and Sparta in ancient Greece, providing insights into the political and military strategies of the time.

3. "The Republic" by Plato - This philosophical work is a cornerstone of Western philosophy and covers topics such as justice, politics, and the nature of the ideal state.

4. "Nicomachean Ethics" by Aristotle - This book is a foundational work in moral philosophy, discussing virtues, happiness, and the good life.

5. "The Iliad" and "The Odyssey" by Homer - These epic po

In [10]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: I assume you're asking about one of the books I mentioned earlier. Please let me know which book you'd like more information on, and I'll be happy to provide a brief summary.


In [11]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hello! I'd be happy to help you find some book suggestions. Can you please tell me what genre or specific topics you're interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Great! Based on your interests in history, philosophy, and Greece, I have a few book suggestions for you:

1. "The Histories" by Herodotus - This classic work is often considered the first work of history in Western literature. It covers the history of the ancient world, including Greece, Egypt, and Persia.

2. "The Peloponnesian War" by Thucydides - This historical account covers the conflict between Athens and Sparta in ancient Greece, providing insights into the political and military strategies of the time.

3. "The Republic" by Plato - This philosophical work is a cornerstone of Western philosophy and covers topics such as justice, politics, and the nature of the ideal state.

4. "Nicomach